In [ ]:
#one to one RNN model

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

import sklearn.datasets as load_digits

from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks, regularizers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, accuracy_score, f1_score

In [ ]:
#Plot Function
def plot_metrics(metric_history):
  metrics = ['loss', 'accuracy', 'precision', 'recall', 'f1_score']

  plt.figure(figsize=(20,10))

  for i, metric in enumerate(metrics, 1):
    plt.subplot(2, 3, i)
    plt.plot(metric_history[metric], label='Train'+metric)
    plt.plot(metric_history[f'val_{metric}'], label='Valid'+metric)
    plt.xlabel('Epochs')
    plt.ylabel(metric.capitalize())
    plt.title(f'{metric.capitalize()} over epochs')
    plt.grid(True)
    plt.legend()
  plt.tight_layout()
  plt.show()

In [ ]:
from sklearn.datasets import load_digits

In [ ]:
#Load and Preprocess Data
digits = load_digits()
X = digits.data
y = digits.target
num_classes = len(np.unique(y))

scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.reshape(X.shape[0], 1, X.shape[1])  #RNN Input

In [ ]:
#Starified Split 80:10:10
s1 = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, temp_idx = next(s1.split(X, y))
X_train, X_temp = X[train_idx], X[temp_idx]
y_train, y_temp = y[train_idx], y[temp_idx]

s2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
train_idx, test_idx = next(s2.split(X_temp, y_temp))
X_val, X_test = X[train_idx], X[test_idx]
y_val, y_test = y[train_idx], y[test_idx]

In [ ]:
#Model

model = models.Sequential([
    layers.SimpleRNN(64, input_shape=(1, X.shape[2]), activation='relu', kernel_regularizer=regularizers.l2(1e5)), #64 = number of rnn neurons
    layers.Dense(num_classes, activation='softmax') ])

lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9 )

optimizer = optimizers.Adam(learning_rate=lr_schedule)

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) #monitoring validation loss
#will stop training if loss inc for 5 times

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
metrics_history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []}

epochs = 50
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=1,
                        batch_size=32,
                        verbose=1)

    # Log metrics
    logs = {k: v[0] for k, v in history.history.items()}  # Ensure values exist
    for k in logs:
        metrics_history[k].append(logs[k])

    # Predictions
    y_train_pred = np.argmax(model.predict(X_train, verbose=0), axis=1)
    y_val_pred = np.argmax(model.predict(X_val, verbose=0), axis=1)

    # Classification Reports
    print(f"\nEpoch {epoch + 1} - Training Metrics:")
    print(classification_report(y_train, y_train_pred, digits=4))

    print(f"\nEpoch {epoch + 1} - Validation Metrics:")
    print(classification_report(y_val, y_val_pred, digits=4))

    # Early stopping check
    if early_stopping.stopped_epoch > 0:
        break


#FINAL CONFUSION METRICS
y_train_pred = np.argmax(model.predict(X_train, verbose=0), axis=1)
y_val_pred = np.argmax(model.predict(X_val, verbose = 0), axis=1)
y_test_pred = np.argmax(model.predict(X_test, verbose = 0), axis=1)


print("\nFinal Confusion Matrix - Train: ")
print(confusion_matrix(y_train, y_train_pred))

print("\nFinal Confusion Matrix - Validation: ")
print(confusion_matrix(y_val, y_val_pred))

print("\nFinal Confusion Matrix - Test: ")
print(confusion_matrix(y_test, y_test_pred))


Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0999 - loss: 5440438.5000 - val_accuracy: 0.2611 - val_loss: 3245642.7500

Epoch 1 - Training Metrics:
              precision    recall  f1-score   support

           0     0.4776    0.2254    0.3062       142
           1     0.2024    0.2329    0.2166       146
           2     0.2207    0.2254    0.2230       142
           3     0.4483    0.0890    0.1486       146
           4     0.0714    0.0552    0.0623       145
           5     0.5083    0.4207    0.4604       145
           6     0.3394    0.5103    0.4077       145
           7     0.2996    0.5804    0.3952       143
           8     0.1784    0.2734    0.2159       139
           9     0.1477    0.0903    0.1121       144

    accuracy                         0.2700      1437
   macro avg     0.2894    0.2703    0.2548      1437
weighted avg     0.2897    0.2700    0.2547      1437


Epoch 1 - Validation Metrics:
              precision    recall  f1-sco

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4195 - loss: 1599.1406 - val_accuracy: 0.3778 - val_loss: 624.4721

Epoch 10 - Training Metrics:
              precision    recall  f1-score   support

           0     1.0000    0.1127    0.2025       142
           1     0.4559    0.8151    0.5848       146
           2     1.0000    0.1127    0.2025       142
           3     0.4674    0.5890    0.5212       146
           4     0.7727    0.1172    0.2036       145
           5     0.8611    0.4276    0.5714       145
           6     0.2645    0.9724    0.4159       145
           7     0.3909    0.9021    0.5455       143
           8     0.0000    0.0000    0.0000       139
           9     0.6667    0.0139    0.0272       144

    accuracy                         0.4092      1437
   macro avg     0.5879    0.4063    0.3275      1437
weighted avg     0.5887    0.4092    0.3296      1437


Epoch 10 - Validation Metrics:
              precision    recall  f1-score   support

    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3687 - loss: 474.5147 - val_accuracy: 0.2944 - val_loss: 176.7506

Epoch 11 - Training Metrics:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       142
           1     0.2733    0.9452    0.4240       146
           2     0.9333    0.0986    0.1783       142
           3     0.4012    0.4589    0.4281       146
           4     0.5233    0.3103    0.3896       145
           5     0.2703    0.9172    0.4176       145
           6     0.4419    0.3931    0.4161       145
           7     0.8605    0.2587    0.3978       143
           8     0.0000    0.0000    0.0000       139
           9     0.0000    0.0000    0.0000       144

    accuracy                         0.3417      1437
   macro avg     0.3704    0.3382    0.2652      1437
weighted avg     0.3710    0.3417    0.2672      1437


Epoch 11 - Validation Metrics:
              precision    recall  f1-score   support

     

...

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1130 - loss: 2.3061 - val_accuracy: 0.1056 - val_loss: 2.3042

Epoch 49 - Training Metrics:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       142
           1     0.0000    0.0000    0.0000       146
           2     0.0000    0.0000    0.0000       142
           3     0.0000    0.0000    0.0000       146
           4     0.0000    0.0000    0.0000       145
           5     0.1009    1.0000    0.1833       145
           6     0.0000    0.0000    0.0000       145
           7     0.0000    0.0000    0.0000       143
           8     0.0000    0.0000    0.0000       139
           9     0.0000    0.0000    0.0000       144

    accuracy                         0.1009      1437
   macro avg     0.0101    0.1000    0.0183      1437
weighted avg     0.0102    0.1009    0.0185      1437


Epoch 49 - Validation Metrics:
              precision    recall  f1-score   support

         

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0992 - loss: 2.3039 - val_accuracy: 0.0889 - val_loss: 2.3046

Epoch 50 - Training Metrics:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       142
           1     0.0000    0.0000    0.0000       146
           2     0.0000    0.0000    0.0000       142
           3     0.1016    1.0000    0.1845       146
           4     0.0000    0.0000    0.0000       145
           5     0.0000    0.0000    0.0000       145
           6     0.0000    0.0000    0.0000       145
           7     0.0000    0.0000    0.0000       143
           8     0.0000    0.0000    0.0000       139
           9     0.0000    0.0000    0.0000       144

    accuracy                         0.1016      1437
   macro avg     0.0102    0.1000    0.0184      1437
weighted avg     0.0103    0.1016    0.0187      1437


Epoch 50 - Validation Metrics:
              precision    recall  f1-score   support

         

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m


Final Confusion Matrix - Train: 
[[  0   0   0 142   0   0   0   0   0   0]
 [  0   0   0 146   0   0   0   0   0   0]
 [  0   0   0 142   0   0   0   0   0   0]
 [  0   0   0 146   0   0   0   0   0   0]
 [  0   0   0 145   0   0   0   0   0   0]
 [  0   0   0 145   0   0   0   0   0   0]
 [  0   0   0 145   0   0   0   0   0   0]
 [  0   0   0 143   0   0   0   0   0   0]
 [  0   0   0 139   0   0   0   0   0   0]
 [  0   0   0 144   0   0   0   0   0   0]]

Final Confusion Matrix - Validation: 
[[ 0  0  0 21  0  0  0  0  0  0]
 [ 0  0  0 14  0  0  0  0  0  0]
 [ 0  0  0 20  0  0  0  0  0  0]
 [ 0  0  0 16  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  0  0  0  0]
 [ 0  0  0 19  0  0  0  0  0  0]
 [ 0  0  0 20  0  0  0  0  0  0]
 [ 0  0  0 19  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  0  0  0  0]
 [ 0  0  0 15  0  0  0  0  0  0]]

Final Confusion Matrix - Test: 
[[ 0  0  0 17  0  0  0  0  0  0]
 [ 0  0  0 24  0  0  0  0  0  0]
 [ 0  0  0 16  0  0  0  0  0  0]
 [ 0  0  0 23  0  0  0  0  0  0]
